In [1]:
# You only need to run this once per machine
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U datasets scipy ipywidgets
!pip install -q -U trl

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cuml 23.8.0 requires dask==2023.7.1, but you have dask 2023.11.0 which is incompatible.
cuml 23.8.0 requires distributed==2023.7.1, but you have distributed 2023.11.0 which is incompatible.
libpysal 4.9.2 requires packaging>=22, but you have packaging 21.3 which is incompatible.
libpysal 4.9.2 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
momepy 0.6.0 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
pymc3 3.11.5 requires numpy<1.22.2,>=1.15.0, but you have numpy 1.24.3 which is incompatible.
pymc3 3.11.5 requires scipy<1.8.0,>=1.7.3, but you have scipy 1.11.4 which is incompatible.
tensorflowjs 4.13.0 requires packaging~=23.1, but you have packaging 21.3 which is incompatible.
ydata-pro

In [2]:
import os
import torch
import pandas as pd
import numpy as np

from transformers import AutoTokenizer, TrainingArguments, AutoModelForCausalLM, BitsAndBytesConfig
from datasets import load_dataset, Dataset
from trl import DPOTrainer
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
import bitsandbytes as bnb

<frozen importlib._bootstrap>:241: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
/opt/conda/lib/python3.10/site-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [3]:
def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])

    return list(lora_module_names)


def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
    if param.requires_grad:
          trainable_params += param.numel()
    print(
      f"trainable params: {trainable_params} || all params: {all_param} || trainables%: {100 * trainable_params / all_param}"
    )

In [4]:
output_dir="./dpo_results"
model_name = "codellama/CodeLlama-7b-Instruct-hf"

In [5]:
train_df = pd.read_csv("/kaggle/input/xlcost-python-dpo/train.csv")
train_df.head()

,id,prompt,accepted_code,rejected_code
0,306,<s>[INST]<<SYS>>\nProvide answers in Python. W...,import math\n\n\ndef findSum(n):\n return m...,import math\n\n\ndef findSum(n):\n return (...
1,307,<s>[INST]<<SYS>>\nProvide answers in Python. W...,"def sum(x, y, n):\n sum1 = ((x ** 2) * (x *...","def sum(x, y, n):\n return sum([x * (x + y)..."
2,313,<s>[INST]<<SYS>>\nProvide answers in Python. W...,dp = [[-1 for j in range(500)]for i in range(5...,dp = [[-1 for j in range(500)]for i in range(5...
3,319,<s>[INST]<<SYS>>\nProvide answers in Python. W...,import sys\n\n\ndef maximumOccurrence(s):\n ...,import sys\n\n\ndef maximumOccurrence(s):\n ...
4,320,<s>[INST]<<SYS>>\nProvide answers in Python. W...,mod = 1000000007\ndp = [[[-1 for i in range(2)...,mod = 1000000007\ndp = [[[-1 for i in range(2)...


In [6]:
train_dataset = Dataset.from_pandas(train_df)

/opt/conda/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


In [7]:
train_dataset

Dataset({
    features: ['id', 'prompt', 'accepted_code', 'rejected_code'],
    num_rows: 240
})

In [8]:
compute_dtype = getattr(torch, "float16")

In [9]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype
)

In [10]:
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config)
model.config.use_cache = False
model = prepare_model_for_kbit_training(model)

config.json:   0%|          | 0.00/646 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [11]:
model_ref = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [12]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

In [13]:
def return_prompt_and_responses(samples):
    return {
        "prompt": samples["prompt"],
        "chosen": samples["accepted_code"],
        "rejected": samples["rejected_code"],
    }

In [14]:
original_columns = train_dataset.column_names
original_columns

['id', 'prompt', 'accepted_code', 'rejected_code']

In [15]:
train_dataset = train_dataset.map(
    return_prompt_and_responses,
    batched=True,
    remove_columns=original_columns
)

Map:   0%|          | 0/240 [00:00<?, ? examples/s]

In [16]:
training_args = TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    gradient_checkpointing =True,
    max_grad_norm= 0.3,
    num_train_epochs=2,
    save_steps= 100,
    learning_rate=2e-4,
    bf16=False,
    save_total_limit=3,
    logging_steps=10,
    output_dir=output_dir,
    optim="paged_adamw_32bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    remove_unused_columns=False,
    report_to="none"
)

In [17]:
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=find_all_linear_names(model),
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

In [18]:
model = get_peft_model(model, peft_config)
print_trainable_parameters(model)

trainable params: 0 || all params: 3520532480 || trainables%: 0.0


In [19]:
dpo_trainer = DPOTrainer(
    model,
    model_ref,
    args=training_args,
    beta=0.1,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    max_prompt_length=1024,
    max_length=2048,
)

In [20]:
dpo_trainer.train()
dpo_trainer.save_model(output_dir)

Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
10,0.326900
20,0.014900
30,0.004400
40,0.000200
50,0.000000
60,0.000100
70,0.000000
80,0.000000
90,0.000000
100,0.000100
